<a href="https://colab.research.google.com/github/uazech/data-mining-velo/blob/master/script/projet_velo_sujet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet

Le projet consiste à prévoir le nombre de vélos loués à chaque heure dans des bornes libres-services de la ville (système type Vélib'). La variable cible est ici la variable **count**.

Voici un descriptif de l'ensemble des variables :
* *datetime* - date et heure du relevé 
* *season* - 1 = printemps , 2 = été, 3 = automne, 4 = hiver 
* *holiday* – indique si le jour est un jour de vacances scolaires 
* *workingday* - indique si le jour est travaillé (ni week-end ni vacances) 
* *weather* - 1: Dégagé à nuageux, 2 : Brouillard, 3 : Légère pluie ou neige, 4 : Fortes averses ou neiges 
* *temp* – température en degrés Celsius 
* *atemp* – température ressentie en degrés Celsius 
* *humidity* – taux d’humidité 
* *windspeed* – vitesse du vent 
* *casual* - nombre de locations d’usagers non abonnés 
* *registered* – nombre de locations d’usagers abonnés 
* *count* – nombre total de locations de vélos 

L'objectif du projet est de mener à bien la création d'un modèle qui pourrait théoriquement être déployé en production. Les étapes d'exploration des données, de traitement et de preprocessing ne sont bien entendu pas à négliger. Il ne s'agit pas d'une compétition de type *Kaggle*, le projet ne sera pas uniquement noté sur la performance du modèle, mais plutôt sur votre approche complète et la justification de chacun de vos choix.

Comme vu durant le cours, soyez faites attention à certains points :
- quel type de problème dois-je traiter ?
- feature engineering : est-ce que j'utilise les données correctement, toutes les données ?
- data leakage : est-ce qu'une de mes features n'est pas trop explicative ?
- ai-je bien traité toutes les données correctement ?
- est-ce que mon modèle est adapté ?
- etc, etc, etc

Soyez vigilant à expliquer et justifier votre démarche à l'aide de visualisation, de commentaires dans vos codes (pensez aux cellules markdown), etc

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Lecture et préparation des données

##Lecture des données

Chargement du fichier velo.csv et affichage des données

In [2]:
url = "https://raw.githubusercontent.com/uazech/data-mining-velo/master/data/input/velo.csv"
df = pd.read_csv(url)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


Affichage du nombre unique de données par colonne. Permet de vérifier qu'il n'y a pas d'anomalies dans les données. 

In [12]:
df.nunique()

datetime      10886
season            4
holiday           2
workingday        2
weather           4
temp             49
atemp            60
humidity         89
windspeed        28
casual          309
registered      731
count           822
dtype: int64

Affichage du nombre de valeur manquante par colonne, afin de vérifier si on doit les traiter ou non.

Il n'y a pas de valeur manquante.

In [10]:
df.isna().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

Affichage des types de chaque feature. On remarque que : 
* La variable ```season``` est une variable catégorielle, et ne devrait pas être de type entier
* La variable ```datetime``` est considérée comme une variable catégorielle, à convertir en timeserie
* La variable ```weather``` est considérée comme une variable entière, alors qu'elle devrait être de type catégorielle
* 

In [4]:
df.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

Deux variables semblent suspectes. En effet, dans l'affichage du head précédent, la target était égale à la somme des deux variable registred et count. Une telle correlation serait préjudiciable. 

Dans la description des données, ces deux variables semblent également liées.

In [28]:
df["tmp"] = df["registered"] + df["casual"]
(df[df["count"] == df["tmp"]].count()) == (df.count())

datetime      True
season        True
holiday       True
workingday    True
weather       True
atemp         True
humidity      True
windspeed     True
casual        True
registered    True
count         True
tmp           True
dtype: bool

Les données sont effectivement liées. Les colonnes casual et registred seront à supprimer lors de la préparation des données. 

## Préparation des données

Changement du type des colonnes

In [0]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [0]:
# df = df.drop("temp", axis=1)